### PRODUCT CLUSTERING ANALYSIS - COMPLETE PIPELINE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
import matplotlib
matplotlib.use('TkAgg')  
import matplotlib.pyplot as plt

In [4]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from os import getenv

In [5]:
load_dotenv(".env")
engine = create_engine(getenv("DB_URL"))

In [6]:
df = pd.read_sql("SELECT * FROM sales219", engine)

### 1. DATA CLEANING & PREPARATION

In [7]:
df.shape

(3678038, 17)

In [8]:
df.columns

Index(['TRANSACTION_RK', 'CALENDAR_DT', 'date', 'time', 'TRANSACTION_TM',
       'ITEM_SK', 'RETAIL_OUTLET_LOCATION_SK', 'POS_TERMINAL_NO', 'CASHIER_NO',
       'ITEM_QTY', 'ITEM_WEIGHT', 'SALES_UOM_CD', 'SELLING_RETAIL_AMT',
       'PROMO_SALES_IND_CD', 'STAPLE_ITEM_FLG', 'REGION_CD', 'CUSTOMER_SK'],
      dtype='object')

In [9]:
df.head()

,TRANSACTION_RK,CALENDAR_DT,date,time,TRANSACTION_TM,ITEM_SK,RETAIL_OUTLET_LOCATION_SK,POS_TERMINAL_NO,CASHIER_NO,ITEM_QTY,ITEM_WEIGHT,SALES_UOM_CD,SELLING_RETAIL_AMT,PROMO_SALES_IND_CD,STAPLE_ITEM_FLG,REGION_CD,CUSTOMER_SK
0,6711099404,01JAN2015,2015-01-01,0 days 22:51:50,22:51:50,12519213,219,7,116,1,0.0,U,6.99,R,Y,4,1
1,6712905826,02JAN2015,2015-01-02,0 days 14:10:33,14:10:33,11838638,219,6,260,1,0.0,U,3.09,R,Y,4,1
2,6712906309,02JAN2015,2015-01-02,0 days 16:48:32,16:48:32,11627223,219,9,212,1,0.0,U,5.49,R,N,4,1
3,6712907445,02JAN2015,2015-01-02,0 days 20:01:05,20:01:05,11840452,219,6,132,1,0.0,U,1.25,N,N,4,1
4,6712905166,02JAN2015,2015-01-02,0 days 07:38:03,7:38:03,14199100,219,6,121,1,0.0,N,4.99,R,N,4,1


In [10]:
# Make a copy to preserve original data
df_clean = df.copy()

In [11]:
# Check for missing values
print("Missing values per column:")
missing_data = df_clean.isnull().sum()
print(missing_data[missing_data > 0])

Missing values per column:
Series([], dtype: int64)


In [12]:
df.dtypes

TRANSACTION_RK                        object
CALENDAR_DT                           object
date                                  object
time                         timedelta64[ns]
TRANSACTION_TM                        object
ITEM_SK                               object
RETAIL_OUTLET_LOCATION_SK              int64
POS_TERMINAL_NO                        int64
CASHIER_NO                             int64
ITEM_QTY                               int64
ITEM_WEIGHT                          float64
SALES_UOM_CD                          object
SELLING_RETAIL_AMT                   float64
PROMO_SALES_IND_CD                    object
STAPLE_ITEM_FLG                       object
REGION_CD                              int64
CUSTOMER_SK                           object
dtype: object

In [13]:
# Ensure data types are correct
df_clean['date'] = pd.to_datetime(df_clean['date'])

In [14]:
df_clean.dtypes

TRANSACTION_RK                        object
CALENDAR_DT                           object
date                          datetime64[ns]
time                         timedelta64[ns]
TRANSACTION_TM                        object
ITEM_SK                               object
RETAIL_OUTLET_LOCATION_SK              int64
POS_TERMINAL_NO                        int64
CASHIER_NO                             int64
ITEM_QTY                               int64
ITEM_WEIGHT                          float64
SALES_UOM_CD                          object
SELLING_RETAIL_AMT                   float64
PROMO_SALES_IND_CD                    object
STAPLE_ITEM_FLG                       object
REGION_CD                              int64
CUSTOMER_SK                           object
dtype: object

In [15]:

# See examples of rows with ITEM_QTY <= 0
print("\n=== Sample rows with ITEM_QTY <= 0 ===")
print(df_clean[df_clean['ITEM_QTY'] <= 0].head(10))

# See examples of rows with SELLING_RETAIL_AMT <= 0
print("\n=== Sample rows with SELLING_RETAIL_AMT <= 0 ===")
print(df_clean[df_clean['SELLING_RETAIL_AMT'] <= 0].head(10))

# See examples of rows with null CUSTOMER_SK
print("\n=== Sample rows with null CUSTOMER_SK ===")
print(df_clean[df_clean['CUSTOMER_SK'].isna()].head(10))


=== Sample rows with ITEM_QTY <= 0 ===
    TRANSACTION_RK CALENDAR_DT       date            time TRANSACTION_TM  \
60      6712905552   02JAN2015 2015-01-02 0 days 12:32:20       12:32:20   
99      6712905896   02JAN2015 2015-01-02 0 days 15:00:22       15:00:22   
108     6711099403   01JAN2015 2015-01-01 0 days 21:05:05       21:05:05   
304     6712928401   02JAN2015 2015-01-02 0 days 09:34:12        9:34:12   
539     6712905697   02JAN2015 2015-01-02 0 days 13:52:35       13:52:35   
588     6712905889   02JAN2015 2015-01-02 0 days 14:54:30       14:54:30   
674     6712929693   02JAN2015 2015-01-02 0 days 18:27:06       18:27:06   
751     6713007233   02JAN2015 2015-01-02 0 days 12:53:39       12:53:39   
785     6712907773   02JAN2015 2015-01-02 0 days 21:26:18       21:26:18   
804     6713007071   02JAN2015 2015-01-02 0 days 10:53:38       10:53:38   

      ITEM_SK  RETAIL_OUTLET_LOCATION_SK  POS_TERMINAL_NO  CASHIER_NO  \
60   11743876                        219          

In [16]:
# Filter out invalid transactions
initial_count = len(df_clean)
df_clean = df_clean[
    (df_clean['ITEM_QTY'] > 0) & 
    (df_clean['SELLING_RETAIL_AMT'] > 0)
]

In [17]:
print(f"Rows after filtering invalid transactions: {len(df_clean)}")
print(f"Removed {initial_count - len(df_clean)} invalid rows")

Rows after filtering invalid transactions: 3482091
Removed 195947 invalid rows


In [18]:
# Calculate line total revenue
df_clean['line_revenue'] = df_clean['ITEM_QTY'] * df_clean['SELLING_RETAIL_AMT']

### 3. FEATURE ENGINEERING

In [19]:
# Define what constitutes a "visit" - using TRANSACTION_RK as visit identifier
df_clean['visit_id'] = df_clean['TRANSACTION_RK']

In [20]:
# Create product-level features
product_features = df_clean.groupby('ITEM_SK').agg({
    'CUSTOMER_SK': ['nunique'],                    # Distinct customers
    'line_revenue': ['sum', 'mean'],              # Total revenue and avg per transaction line
    'visit_id': ['nunique'],                      # Number of visits
    'ITEM_QTY': ['sum', 'mean'],                  # Total quantity and avg quantity
    'date': ['min', 'max', 'nunique']             # First/last sale date, unique days
}).reset_index()

In [21]:
# Flatten column names
product_features.columns = [
    'product_id',
    'distinct_customers',
    'revenue',
    'avg_line_revenue',
    'number_of_visits',
    'total_quantity',
    'avg_quantity_per_line',
    'first_sale_date',
    'last_sale_date',
    'unique_sale_days'
]

In [22]:
# Calculate derived features
product_features['avg_revenue_per_visit'] = product_features['revenue'] / product_features['number_of_visits']
product_features['avg_revenue_per_customer'] = product_features['revenue'] / product_features['distinct_customers']
product_features['purchase_frequency'] = product_features['number_of_visits'] / product_features['distinct_customers']

In [23]:
# Calculate recency (days since last sale, assuming analysis date is end of 2019)
analysis_date = pd.Timestamp('2026-01-15')
product_features['days_since_last_sale'] = (analysis_date - product_features['last_sale_date'])
product_features['sales_duration_days'] = (product_features['last_sale_date'] - product_features['first_sale_date']).dt.days + 1

In [24]:
# Calculate additional useful metrics
product_features['avg_daily_revenue'] = product_features['revenue'] / product_features['sales_duration_days']
product_features['avg_visit_value'] = product_features['revenue'] / product_features['number_of_visits']
product_features['customer_penetration_rate'] = product_features['distinct_customers'] / product_features['number_of_visits']

In [25]:
print(f"Created features for {len(product_features)} unique products")
print("\nFeature Statistics:")
print(product_features.describe().T[['mean', 'std', 'min', 'max']])

Created features for 30507 unique products

Feature Statistics:
                                                    mean  \
distinct_customers                             53.873308   
revenue                                       575.240731   
avg_line_revenue                                6.420868   
number_of_visits                              114.140722   
total_quantity                                134.906382   
avg_quantity_per_line                           1.153339   
first_sale_date            2015-02-10 21:09:41.650113024   
last_sale_date             2015-08-05 20:34:40.204543232   
unique_sale_days                               45.909136   
avg_revenue_per_visit                           6.420868   
avg_revenue_per_customer                       11.205245   
purchase_frequency                              1.763539   
days_since_last_sale        3815 days 03:25:19.795456768   
sales_duration_days                           176.975678   
avg_daily_revenue                   

### 4. DATA QUALITY CHECK & OUTLIER DETECTION

In [26]:
# Check for infinite or null values
print("Checking for data issues:")
print(f"Rows with NaN values: {product_features.isnull().sum().sum()}")
print(f"Rows with infinite values: {np.isinf(product_features.select_dtypes(include=[np.number])).sum().sum()}")

Checking for data issues:
Rows with NaN values: 0
Rows with infinite values: 0


In [27]:
# Identify extreme outliers using IQR method
def detect_outliers_iqr(df, column, threshold=3):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

In [28]:
outlier_report = {}
print("\nOutlier Detection (using IQR method):")
for col in ['revenue', 'distinct_customers', 'number_of_visits']:
    outliers, lower, upper = detect_outliers_iqr(product_features, col, threshold=3)
    outlier_report[col] = {
        'outlier_count': len(outliers),
        'percentage': len(outliers) / len(product_features) * 100,
        'lower_bound': lower,
        'upper_bound': upper
    }
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(product_features)*100:.1f}%)")


Outlier Detection (using IQR method):
revenue: 1994 outliers (6.5%)
distinct_customers: 1838 outliers (6.0%)
number_of_visits: 1953 outliers (6.4%)


In [29]:
# Create a flag for extreme outliers
product_features['is_extreme_outlier'] = False
for col in ['revenue', 'distinct_customers']:
    outliers, _, _ = detect_outliers_iqr(product_features, col, threshold=5)  # More strict for extreme
    product_features.loc[outliers.index, 'is_extreme_outlier'] = True

print(f"\nExtreme outliers (across all metrics): {product_features['is_extreme_outlier'].sum()} products")


Extreme outliers (across all metrics): 1416 products


### 5. FEATURE SELECTION & PREPROCESSING

In [30]:
# Select features for clustering
selected_features = [
    'distinct_customers',
    'revenue', 
    'number_of_visits',
    'avg_revenue_per_visit',
    'avg_revenue_per_customer',
    'purchase_frequency',
    'total_quantity',
    'unique_sale_days',
    'avg_daily_revenue'
]

In [31]:
# Create feature matrix
X = product_features[selected_features].copy()

In [38]:
# Handle zeros (log transformation will fail with zeros)
# Add small constant to enable log transform
X_log = X.copy()
for col in X_log.columns:
    if X_log[col].min() == 0:
        print(f"Adding small constant to column '{col}' to handle zeros for log transform.")
        X_log[col] = X_log[col] + 1e-10

In [33]:
# Apply log transformation to reduce skewness
X_log_transformed = np.log1p(X_log)  # log(1+x) to handle zeros

In [34]:
# Scale the features
scaler = RobustScaler()  # Robust to outliers
X_scaled = scaler.fit_transform(X_log_transformed)

In [35]:
print(f"Selected {len(selected_features)} features for clustering:")
for i, feat in enumerate(selected_features, 1):
    print(f"{i:2}. {feat}")

Selected 9 features for clustering:
 1. distinct_customers
 2. revenue
 3. number_of_visits
 4. avg_revenue_per_visit
 5. avg_revenue_per_customer
 6. purchase_frequency
 7. total_quantity
 8. unique_sale_days
 9. avg_daily_revenue


In [41]:
X_scaled

array([[-0.58851158, -0.76690271, -0.51759191, ..., -0.55571004,
        -0.51693294, -0.13217135],
       [-0.83416678, -0.85512381, -0.92283161, ..., -0.9581008 ,
        -0.97844512, -0.64849995],
       [-0.83416678, -1.39241255, -0.92283161, ..., -0.9581008 ,
        -0.97844512, -0.71544571],
       ...,
       [-1.02915428, -1.38464033, -1.09035369, ..., -1.12444515,
        -1.16922969,  0.95761523],
       [-1.02915428, -1.59386786, -1.09035369, ..., -1.12444515,
        -1.16922969,  0.50899691],
       [-1.02915428, -1.57499439, -1.09035369, ..., -1.12444515,
        -1.16922969,  0.54946475]], shape=(30507, 9))

In [42]:
# Check basic statistics
print("Mean of each column:", X_scaled.mean(axis=0).round(3))
print("Std of each column:", X_scaled.std(axis=0).round(3))
print("\nMin values per column:", X_scaled.min(axis=0).round(3))
print("Max values per column:", X_scaled.max(axis=0).round(3))
print("Median of each column:", np.median(X_scaled, axis=0).round(3))

Mean of each column: [ 0.034 -0.02   0.014  0.034 -0.003  0.037  0.015 -0.037  0.22 ]
Std of each column: [0.672 0.718 0.659 0.672 0.808 0.867 0.671 0.629 0.766]

Min values per column: [-1.029 -2.096 -1.09  -2.643 -3.293 -1.066 -1.124 -1.169 -0.736]
Max values per column: [ 3.305  3.144  3.324  7.953  7.4   14.961  3.268  1.105  5.928]
Median of each column: [0. 0. 0. 0. 0. 0. 0. 0. 0.]


### 6. DETERMINING OPTIMAL NUMBER OF CLUSTERS (K)

In [ ]:
# Try different values of K
k_range = range(2, 10)
inertias = []
silhouette_scores = []

In [ ]:
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    
    # Silhouette score (skip for k=1 as it's not defined)
    if k > 1:
        silhouette_avg = silhouette_score(X_scaled, cluster_labels)
        silhouette_scores.append(silhouette_avg)
    else:
        silhouette_scores.append(0)

In [43]:
# Plot elbow curve and silhouette scores
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Elbow method
ax1.plot(k_range, inertias, 'bo-', linewidth=2)
ax1.set_xlabel('Number of Clusters (K)')
ax1.set_ylabel('Inertia')
ax1.set_title('Elbow Method for Optimal K')
ax1.grid(True)

# Silhouette scores
ax2.plot(k_range[1:], silhouette_scores[1:], 'ro-', linewidth=2)
ax2.set_xlabel('Number of Clusters (K)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Scores for Different K')
ax2.grid(True)

plt.tight_layout()
plt.savefig('k_selection_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [56]:
# Let's use the silhouette-based K 
optimal_k = 6
print(f"\nUsing K = {optimal_k} for clustering")


Using K = 6 for clustering


### 7. RUNNING K-MEANS CLUSTERING

In [57]:
# Perform K-means clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
product_features['cluster'] = kmeans.fit_predict(X_scaled)

In [58]:
# Get cluster centroids in the scaled space
centroids_scaled = kmeans.cluster_centers_

In [59]:
# Transform centroids back to original scale
centroids_log = scaler.inverse_transform(centroids_scaled)
centroids_original = np.expm1(centroids_log)  # Reverse log(1+x)

In [60]:
# Create centroids DataFrame
centroids_df = pd.DataFrame(
    centroids_original,
    columns=selected_features,
    index=[f'Cluster_{i}' for i in range(optimal_k)]
)

In [63]:
centroids_df.head()

,distinct_customers,revenue,number_of_visits,avg_revenue_per_visit,avg_revenue_per_customer,purchase_frequency,total_quantity,unique_sale_days,avg_daily_revenue
Cluster_0,158.416015,1716.341581,328.385059,5.305931,10.935681,2.091759,384.365880,129.908769,8.064733
Cluster_1,5.656369,62.731101,7.749273,8.375275,11.530126,1.394624,8.467676,7.290666,0.614752
Cluster_2,35.962697,250.117704,62.937824,4.035571,7.043420,1.764679,69.799080,48.192055,1.326318
Cluster_3,5.537201,21.814986,7.148220,3.198820,4.107574,1.303848,7.550886,6.479024,0.480756
Cluster_4,14.472980,286.388872,33.424491,8.742833,20.617044,2.415024,40.055607,28.566389,1.778692


In [61]:
print(f"Clustering complete! Products assigned to {optimal_k} clusters.")
print("\nCluster Distribution:")
cluster_dist = product_features['cluster'].value_counts().sort_index()
for cluster, count in cluster_dist.items():
    print(f"Cluster {cluster}: {count} products ({count/len(product_features)*100:.1f}%)")

Clustering complete! Products assigned to 6 clusters.

Cluster Distribution:
Cluster 0: 4669 products (15.3%)
Cluster 1: 6524 products (21.4%)
Cluster 2: 8729 products (28.6%)
Cluster 3: 4417 products (14.5%)
Cluster 4: 3923 products (12.9%)
Cluster 5: 2245 products (7.4%)


### 8. CLUSTER PROFILING & INTERPRETATION

In [64]:
# Calculate cluster statistics
cluster_profiles = product_features.groupby('cluster')[selected_features].agg(['mean', 'std', 'count'])

In [65]:
# Flatten the multi-level columns for easier viewing
cluster_profiles_flat = pd.DataFrame()
for col in selected_features:
    cluster_profiles_flat[f'{col}_mean'] = cluster_profiles[col]['mean']
    cluster_profiles_flat[f'{col}_std'] = cluster_profiles[col]['std']

In [66]:
# Add cluster size
cluster_profiles_flat['product_count'] = cluster_profiles[selected_features[0]]['count']
cluster_profiles_flat['percentage'] = (cluster_profiles_flat['product_count'] / len(product_features)) * 100

In [67]:
print("\nCluster Profiles (Key Metrics):")
profile_cols = ['product_count', 'percentage', 'revenue_mean', 'distinct_customers_mean', 
                'number_of_visits_mean', 'avg_revenue_per_customer_mean']
print(cluster_profiles_flat[profile_cols].round(2))


Cluster Profiles (Key Metrics):
         product_count  percentage  revenue_mean  distinct_customers_mean  \
cluster                                                                     
0                 4669       15.30       2690.85                   237.74   
1                 6524       21.39         79.56                     6.83   
2                 8729       28.61        300.63                    43.39   
3                 4417       14.48         30.65                     6.96   
4                 3923       12.86        425.41                    19.19   
5                 2245        7.36         16.87                     1.87   

         number_of_visits_mean  avg_revenue_per_customer_mean  
cluster                                                        
0                       540.56                          11.96  
1                         9.59                          12.07  
2                        76.48                           7.40  
3                         9.49

### Cluster 0: The Foundation Pillars

Size: 4,669 products (15.3% of portfolio)
Revenue: Revenue powerhouse (10.3x average revenue)
Customer Base: Extremely broad customer base (7.5x average)
Engagement: Exceptionally high engagement (8.3x average visits)
Customer Value: Average customer value (1.0x average)

DESCRIPTION: 
These are your flagship products that form the backbone of your business. 
They attract massive customer traffic (237+ customers each), generate 
significant store visits, and deliver enormous total revenue. While each 
customer spends about average ($11.96), the sheer volume makes these 
products indispensable.

BUSINESS IMPLICATION: Protect these products at all costs. Ensure 
excellent availability, prominent placement, and consistent quality.


### Cluster 1: The Mass Market Essentials

Size: 6,524 products (21.4% of portfolio) - LARGEST GROUP
Revenue: Low revenue (0.3x average)
Customer Base: Limited customer reach (0.2x average)
Engagement: Very low engagement (0.2x average visits)
Customer Value: Good customer value (1.0x average)

DESCRIPTION: 
This is your largest product category by count. These are everyday items 
that serve a specific, consistent need for a small group of customers 
(~7 customers per product). Each customer who buys these products tends 
to spend appropriately ($12.07), but the limited customer base keeps 
total revenue low.

BUSINESS IMPLICATION: These are stable but unspectacular performers. 
Consider bundling them with higher-margin items or using them to 
encourage repeat visits from their loyal (but small) customer base.


### Cluster 1: The Mass Market Essentials

Size: 6,524 products (21.4% of portfolio) - LARGEST GROUP
Revenue: Low revenue (0.3x average)
Customer Base: Limited customer reach (0.2x average)
Engagement: Very low engagement (0.2x average visits)
Customer Value: Good customer value (1.0x average)

DESCRIPTION: 
This is your largest product category by count. These are everyday items 
that serve a specific, consistent need for a small group of customers 
(~7 customers per product). Each customer who buys these products tends 
to spend appropriately ($12.07), but the limited customer base keeps 
total revenue low.

BUSINESS IMPLICATION: These are stable but unspectacular performers. 
Consider bundling them with higher-margin items or using them to 
encourage repeat visits from their loyal (but small) customer base.


### Cluster 3: The Inactive Inventory

Size: 4,417 products (14.5% of portfolio)
Revenue: Minimal revenue contributor (0.1x average)
Customer Base: Limited customer reach (0.2x average)
Engagement: Very low engagement (0.2x average visits)
Customer Value: Low value per customer (0.4x average)

DESCRIPTION:
Troubling cluster with multiple red flags. These products attract 
few customers (~7), generate minimal visits, and those who do buy 
spend very little ($4.38). They're underperforming across all 
metrics and likely represent dead inventory or products that have 
lost relevance.

BUSINESS IMPLICATION: Immediate action required. Consider:
1. Deep discounts to clear inventory
2. Bundling with popular items
3. Complete discontinuation if no strategic value
4. Investigation into why they're failing


### Cluster 4: The Premium Niche

Size: 3,923 products (12.9% of portfolio)
Revenue: High-revenue (1.5x average)
Customer Base: Limited customer reach (0.5x average)
Engagement: Moderate engagement (0.7x average visits)
Customer Value: High value per customer (2.2x average)

DESCRIPTION:
A highly interesting cluster. While these products have a relatively 
small customer base (~19 customers), those customers spend 
significantly more ($26.32 each) and generate solid total revenue. 
This suggests higher-priced items or products with strong value 
proposition for a specific customer segment.

BUSINESS IMPLICATION: Target marketing to high-value customer 
segments. These products have excellent revenue per customer - 
focus on customer retention and upselling within this group.


### Cluster 5: The Neglected Opportunities

Size: 2,245 products (7.4% of portfolio) - SMALLEST GROUP
Revenue: Minimal revenue contributor (0.1x average)
Customer Base: Niche customer segment (0.1x average)
Engagement: Extremely low engagement (0.0x average visits)
Customer Value: Average customer value (0.8x average)

DESCRIPTION:
The smallest and least engaged product group. With only ~2 customers 
per product and almost no store visits, these are essentially 
invisible in your portfolio. However, the customers who do find 
them spend reasonably well ($8.93), suggesting there might be 
untapped potential if they were better marketed or displayed.

BUSINESS IMPLICATION: Two possible strategies:
1. Test improved merchandising and marketing
2. If no improvement, discontinue to simplify inventory

### 9. OUTLIER ANALYSIS & EXPLANATION

In [85]:
# Identify outliers (products in very small clusters or far from centroids)
product_features['distance_to_centroid'] = np.linalg.norm(
    X_scaled - centroids_scaled[product_features['cluster']], axis=1
)

In [88]:
# Flag statistical outliers based on distance to centroid
distance_q75 = product_features['distance_to_centroid'].quantile(0.75)
distance_q25 = product_features['distance_to_centroid'].quantile(0.25)
distance_iqr = distance_q75 - distance_q25
outlier_threshold = distance_q75 + 1.5 * distance_iqr

In [89]:
product_features['is_distance_outlier'] = product_features['distance_to_centroid'] > outlier_threshold

In [90]:
# Get top outliers
top_outliers = product_features[
    product_features['is_distance_outlier'] | 
    product_features['is_extreme_outlier']
].sort_values('distance_to_centroid', ascending=False).head(10)

In [91]:
print(f"Identified {len(top_outliers)} significant outliers")
print("\nTop 10 Outliers (products far from their cluster centroids):")
outlier_cols = ['product_id', 'cluster_name', 'revenue', 'distinct_customers', 
                'number_of_visits', 'distance_to_centroid']
print(top_outliers[outlier_cols].round(2))

Identified 10 significant outliers

Top 10 Outliers (products far from their cluster centroids):
      product_id        cluster_name   revenue  distinct_customers  \
29687   15657824  Frequent Purchases  4403.500                   3   
22006   14333565  Frequent Purchases 12847.000                  18   
421     11603103  Frequent Purchases 19563.990                  32   
419     11603093  Frequent Purchases 12778.000                  16   
26143   15072359  Frequent Purchases 12790.490                  14   
5663    11680016       Star Products 84541.680                 290   
25740   15029236  Frequent Purchases  9429.000                  16   
27552   15251644  Frequent Purchases   693.000                   3   
13368   11872825  Frequent Purchases  1300.000                   1   
13366   11872816  Frequent Purchases  1300.000                   1   

       number_of_visits  distance_to_centroid  
29687               394                15.850  
22006              1175             

### 10. VISUALIZATION

In [99]:
# 10.1 Scatter plot of key metrics colored by cluster
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Revenue vs Customers
scatter1 = axes[0,0].scatter(
    product_features['distinct_customers'],
    product_features['revenue'],
    c=product_features['cluster'],
    cmap='tab20',
    alpha=0.6,
    s=50
)
axes[0,0].set_xlabel('Distinct Customers')
axes[0,0].set_ylabel('Revenue')
axes[0,0].set_title('Revenue vs Customer Count by Cluster')
axes[0,0].set_xscale('log')
axes[0,0].set_yscale('log')
plt.colorbar(scatter1, ax=axes[0,0], label='Cluster')

# Plot 2: Revenue per Customer vs Purchase Frequency
scatter2 = axes[0,1].scatter(
    product_features['purchase_frequency'],
    product_features['avg_revenue_per_customer'],
    c=product_features['cluster'],
    cmap='tab20',
    alpha=0.6,
    s=50
)
axes[0,1].set_xlabel('Purchase Frequency')
axes[0,1].set_ylabel('Avg Revenue per Customer')
axes[0,1].set_title('Customer Value vs Loyalty by Cluster')
axes[0,1].set_xscale('log')
axes[0,1].set_yscale('log')

# Plot 3: Cluster distribution
cluster_counts = product_features['cluster_name'].value_counts()
axes[1,0].barh(range(len(cluster_counts)), cluster_counts.values)
axes[1,0].set_yticks(range(len(cluster_counts)))
axes[1,0].set_yticklabels(cluster_counts.index)
axes[1,0].set_xlabel('Number of Products')
axes[1,0].set_title('Product Distribution Across Clusters')
axes[1,0].invert_yaxis()

# Plot 4: Box plot of revenue by cluster
product_features.boxplot(column='revenue', by='cluster_name', ax=axes[1,1])
axes[1,1].set_title('Revenue Distribution by Cluster')
axes[1,1].set_ylabel('Revenue (log scale)')
axes[1,1].set_yscale('log')
axes[1,1].tick_params(axis='x', rotation=45)

plt.suptitle('Product Cluster Analysis', fontsize=16)
plt.tight_layout()
plt.savefig('cluster_visualizations.png', dpi=150, bbox_inches='tight')
plt.show()

In [100]:
# 10.2 Heatmap of cluster centroids
plt.figure(figsize=(12, 8))
centroids_plot = centroids_df.copy()

# Normalize for better visualization
centroids_normalized = centroids_plot.apply(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() > x.min() else x)

sns.heatmap(centroids_normalized.T, annot=True, cmap='YlOrRd', 
            fmt='.2f', linewidths=0.5, cbar_kws={'label': 'Normalized Value'})
plt.title('Cluster Centroids (Normalized)')
plt.xlabel('Cluster')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('cluster_centroids_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

### 11. DELIVERABLES GENERATION

In [101]:
# Deliverable 1: Product Feature Table
product_feature_table = product_features.copy()
print(f"✓ Product Feature Table: {len(product_feature_table)} products with {len(product_feature_table.columns)} features")

✓ Product Feature Table: 30507 products with 23 features


In [102]:
# Deliverable 2: Outlier Explanation
outlier_explanation = top_outliers.copy()
# Add explanation column
def generate_outlier_explanation(row):
    explanations = []
    if row['is_extreme_outlier']:
        explanations.append("Extreme value in key metrics")
    if row['distance_to_centroid'] > outlier_threshold:
        explanations.append("Far from cluster center")
    if row['revenue'] > product_features['revenue'].quantile(0.95):
        explanations.append("Top 5% in revenue")
    if row['distinct_customers'] < product_features['distinct_customers'].quantile(0.05):
        explanations.append("Very few unique customers")
    return "; ".join(explanations) if explanations else "Moderate outlier"

outlier_explanation['explanation'] = outlier_explanation.apply(generate_outlier_explanation, axis=1)
print(f"✓ Outlier Explanation: {len(outlier_explanation)} outliers documented")

✓ Outlier Explanation: 10 outliers documented


In [103]:
# Deliverable 3: Product Cluster Centroids
cluster_centroids = centroids_df.copy()
cluster_centroids['cluster_name'] = [cluster_names[i] for i in range(optimal_k)]
cluster_centroids['product_count'] = cluster_profiles_flat['product_count'].values
print(f"✓ Product Cluster Centroids: {len(cluster_centroids)} clusters")

✓ Product Cluster Centroids: 6 clusters


In [104]:
# Deliverable 4: Product Cluster Profiles
cluster_profiles_detailed = pd.DataFrame()

for cluster in range(optimal_k):
    cluster_data = product_features[product_features['cluster'] == cluster]
    profile = {
        'cluster': cluster,
        'cluster_name': cluster_names[cluster],
        'product_count': len(cluster_data),
        'percentage_of_total': len(cluster_data) / len(product_features) * 100,
        'avg_revenue': cluster_data['revenue'].mean(),
        'avg_customers': cluster_data['distinct_customers'].mean(),
        'avg_visits': cluster_data['number_of_visits'].mean(),
        'avg_revenue_per_customer': cluster_data['avg_revenue_per_customer'].mean(),
        'median_revenue': cluster_data['revenue'].median(),
        'revenue_contribution': cluster_data['revenue'].sum() / product_features['revenue'].sum() * 100,
        'top_product_example': cluster_data.nlargest(1, 'revenue')['product_id'].iloc[0] if len(cluster_data) > 0 else None
    }
    cluster_profiles_detailed = pd.concat([cluster_profiles_detailed, pd.DataFrame([profile])], ignore_index=True)

print(f"✓ Product Cluster Profiles: Detailed profiles for {len(cluster_profiles_detailed)} clusters")

✓ Product Cluster Profiles: Detailed profiles for 6 clusters


### 12. SAVING RESULTS

In [105]:
# Save all deliverables
product_feature_table.to_csv('product_feature_table.csv', index=False)
outlier_explanation.to_csv('outlier_explanation.csv', index=False)
cluster_centroids.to_csv('product_cluster_centroids.csv')
cluster_profiles_detailed.to_csv('product_cluster_profiles.csv', index=False)

In [106]:
# Save the clustered product data
product_features.to_csv('clustered_products.csv', index=False)

In [107]:
print("Files saved:")
print("1. product_feature_table.csv - Complete feature table for all products")
print("2. outlier_explanation.csv - Detailed outlier analysis")
print("3. product_cluster_centroids.csv - Cluster centroids (characteristics)")
print("4. product_cluster_profiles.csv - Business-friendly cluster profiles")
print("5. clustered_products.csv - All products with cluster assignments")

Files saved:
1. product_feature_table.csv - Complete feature table for all products
2. outlier_explanation.csv - Detailed outlier analysis
3. product_cluster_centroids.csv - Cluster centroids (characteristics)
4. product_cluster_profiles.csv - Business-friendly cluster profiles
5. clustered_products.csv - All products with cluster assignments


### 13. EXECUTIVE SUMMARY

In [108]:
total_revenue = product_features['revenue'].sum()
total_products = len(product_features)

In [109]:
print("\n📊 Analysis Overview:")
print(f"   • Total Products Analyzed: {total_products:,}")
print(f"   • Total Revenue: ${total_revenue:,.0f}")
print(f"   • Number of Clusters: {optimal_k}")
print(f"   • Outliers Identified: {len(outlier_explanation)}")


📊 Analysis Overview:
   • Total Products Analyzed: 30,507
   • Total Revenue: $17,548,869
   • Number of Clusters: 6
   • Outliers Identified: 10


In [110]:
print("\n🎯 Cluster Breakdown:")
for _, profile in cluster_profiles_detailed.iterrows():
    print(f"   • {profile['cluster_name']}: {profile['product_count']} products "
          f"({profile['percentage_of_total']:.1f}%), "
          f"Contributes {profile['revenue_contribution']:.1f}% of revenue")


🎯 Cluster Breakdown:
   • Star Products: 4669 products (15.3%), Contributes 71.6% of revenue
   • Low-Performers: 6524 products (21.4%), Contributes 3.0% of revenue
   • Average Products: 8729 products (28.6%), Contributes 15.0% of revenue
   • Low-Performers: 4417 products (14.5%), Contributes 0.8% of revenue
   • Frequent Purchases: 3923 products (12.9%), Contributes 9.5% of revenue
   • Low-Performers: 2245 products (7.4%), Contributes 0.2% of revenue


In [111]:
print("\n🚨 Key Findings:")
# Find the star cluster
star_cluster = cluster_profiles_detailed.loc[cluster_profiles_detailed['avg_revenue'].idxmax()]
print(f"   1. {star_cluster['cluster_name']} are your highest revenue generators")
print(f"      ({star_cluster['product_count']} products, {star_cluster['revenue_contribution']:.1f}% of total revenue)")


🚨 Key Findings:
   1. Star Products are your highest revenue generators
      (4669 products, 71.6% of total revenue)


In [112]:
# Find the most popular cluster (by customers)
popular_cluster = cluster_profiles_detailed.loc[cluster_profiles_detailed['avg_customers'].idxmax()]
print(f"   2. {popular_cluster['cluster_name']} reach the most customers")
print(f"      (Average {popular_cluster['avg_customers']:.0f} customers per product)")

   2. Star Products reach the most customers
      (Average 238 customers per product)


In [113]:
# Find niche products
if 'High-Value Niche' in cluster_names.values():
    niche_cluster = cluster_profiles_detailed[cluster_profiles_detailed['cluster_name'] == 'High-Value Niche']
    if not niche_cluster.empty:
        print("   3. High-Value Niche products have high revenue but fewer customers")
        print("      (Potential for premium pricing or targeted marketing)")

print("\n💡 Recommendations:")
print(f"   1. Focus marketing efforts on {star_cluster['cluster_name']} cluster")
print("   2. Investigate outliers for potential errors or special opportunities")
print("   3. Use cluster assignments for inventory planning and pricing strategies")
print(f"   4. Consider promoting {popular_cluster['cluster_name']} to drive customer acquisition")

print("\n" + "=" * 70)
print("ANALYSIS COMPLETE ✅")
print("=" * 70)


💡 Recommendations:
   1. Focus marketing efforts on Star Products cluster
   2. Investigate outliers for potential errors or special opportunities
   3. Use cluster assignments for inventory planning and pricing strategies
   4. Consider promoting Star Products to drive customer acquisition

ANALYSIS COMPLETE ✅
